# Perfilamiento sobre algoritmo Simplex

### 1. Medición de tiempo

Características de la instancia que utilizamos para el perfilamiento

In [1]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2299.998
BogoMIPS:            4599.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd i

In [2]:
#%%bash
#sudo lshw -C memory

In [3]:
import math
import time
from versiones import Simplex_v0
import numpy as np
from pytest import approx
from scipy.optimize import linprog
from pytest import approx

### Módulo time

Ejemplo Maximización para evualuar tiempo de ejecución

In [4]:
import random

def listaAleatorios(n,m):
      lista = [0]  * n
      for i in range(n):
          lista[i] = random.randint(0, m)
      return lista

In [5]:
n=9000
m=100

In [6]:
c=listaAleatorios(n,1)
b=listaAleatorios(m,1)
A_=listaAleatorios(n*m,1)
A=np.resize(np.array(A_),(m,n))

In [ ]:
start_time = time.time()
problema = Simplex_v0.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Con Scipy

In [ ]:
c_scipu=[ -x for x in c]

start_time = time.time()
opt = linprog(c=c_scipu, A_ub=A, b_ub=b,
              method="simplex")
end_time = time.time()
secs = end_time-start_time
print("Scipy  tomó",secs,"segundos" )

Obs. En esta primera parte nos damos cuenta que el algoritmo implementado es un poco más rápido que el de Scipy.

In [ ]:
method_result== approx(opt.x, abs=1e-6, rel=1e-6)

### Comando Magic %time

In [ ]:
%time problema.solve()

### Cprofile

Para poder visualizar en que secciones de código se tarda más

In [ ]:
import cProfile

In [ ]:
cprof = cProfile.Profile()
cprof.enable()
problema = Simplex_v0.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
cprof.disable()
cprof.print_stats(sort='cumtime')

In [ ]:
import pstats

cprof.dump_stats("Simplex_stats")

In [ ]:
p_simplex_stats = pstats.Stats("Simplex_stats")
print(p_simplex_stats.sort_stats("cumulative").print_stats(10))

In [ ]:
print(p_simplex_stats.sort_stats("cumulative").print_stats("solve|module"))

In [ ]:
#numero de llamadas a funciones primitivas
print(p_simplex_stats.prim_calls)

In [ ]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

In [ ]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callees("print|dot|ndim|solve")

Obs. Se observa que las partes del código que tienen un mayor número de llamadas son

* **print**- El cual se utiliza al final del algoritmo para mostrar los valores óptimos 
* **dot**(producto punto)
* **dim** y **size** que se ocupan al principio del algoritmo para saber la longuitud de las matrices a optimizar
* **solve**- Este es el que tiene mayor número de llamadas

### Lineprofiler

Para saber línea por línea en que parte nuestro algoritmo se está tardano más en ejecutar

In [ ]:
import line_profiler

problema = Simplex_v0.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

In [ ]:
print(line_prof.print_stats())

De acuerdo a las estadísticas resultantes con line profiler se tienen 7 líneas en el código que tienen un porcentaje elevado de tiempo en ejecución:
* Línea 64 - `A = np.c_[A,identity_A]` que es la parte que construye la matriz completa A al agregar la identidad por las variables de holgura
* Línea 81 - `lista.append (-lambda_ + np.dot(nu, A[:, N_list_idx[i]]))` para crear la lista de las lambas a evaluar en el método
* Línea 88 - ` d = np.linalg.solve(B, A[:,idx_x_N]) ` para la solución del problema de ecuaciones
* Línea 97 -  `if np.isnan(lista2).all() == True:`para evular una solución *ounbounded*
* Línea 117 - `nu = np.linalg.solve(B.T, c_B)`solución del problema de ecuaciones
* Línea 137 - `print("Optimization completed successfully !") `para imprimir si la optimización fue correcta


Respecto a las demás líneas, lo que decidimos mejorar fue la línea 63 con otra función de Pyhton: `np.hstack`  y evaluar los resultados.

**Modificando la línea 63 con la funcion `np.hstack`**

In [ ]:
import line_profiler
from versiones import Simplex_v0_1

problema = Simplex_v0_1.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

In [ ]:
print(line_prof.print_stats())

Como se puede observar en las estadísticas de arriba la línea 64 si mejora bastante en las 3 columnas : Time, Per Hit y % Time. Con lo cual se logró reducir el % de tiempo de ejecución, se pasó de 11  a  5.9.

Donde la línea que ocupa más recursos es la 137, únicamente se llama una vez a esa parte del código pero sabemos que los print siempre son computacionalmente costosos, por lo cual podríamos considerar eliminarlo o solo realizar la impresión si el usuario lo desea con un **verbose**. 

**Agregando un `verbose` para los prints de la solución**

In [ ]:
import line_profiler
import Simplex

problema = Simplex.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

In [ ]:
#Con verbose=True
problema = Simplex.Simplex(c,A,b,problem='Max', verbose=True)
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

Evaluado en `line_profiler` con las modificaciones

In [ ]:
problema = Simplex.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

In [ ]:
print(line_prof.print_stats())

De acuerdo a los últimos estadísticos de con la nueva versión del paquete logramos reducir el tiempo que por default acumulaba el print anterior de la solución, ahora le dejamos al usuarios que si desea ver los print's en la ejecución lo haga poniendo el parámetro `verbose=True`, de otra manera podría acceder a los valores simplemente imprimiendo : x_result, opt y status.

**Evalaución del tiempo en segundos con `time` con el último algoritmo:**

In [ ]:
start_time = time.time()
problema = Simplex.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

In [ ]:
#Versión anterior v0
start_time = time.time()
problema = Simplex_v0.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Conclusión:

Si logramos mejorar el tiempo de ejecución del algoritmo y realmente comparado con nuestro benchamark de "Scipy" tiene un buen desempeño en tiempo.